In [23]:
import tensorflow as tf
import keras
from keras import layers, activations, optimizers, losses, metrics, callbacks

In [28]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation=activations.relu),
            layers.Dense(embed_dim)
        ])
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'embed_dim': self.embed_dim,
            'num_heads': self.num_heads,
            'dense_dim': self.dense_dim
        })
        return config

In [31]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = layers.Input(shape=(None,), dtype=tf.int64)
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim=embed_dim, dense_dim=dense_dim, num_heads=2)(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_3 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_3 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 257 

In [32]:
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size
)

max_length = 600
max_tokens = 20000

text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorization = layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length=max_length)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
int_val_ds = val_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)
int_test_ds = test_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=16)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [34]:

callback_list = [
    callbacks.ModelCheckpoint('transformer_encoder', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callback_list)

model = keras.models.load_model('transformer_encoder', custom_objects={'TransformerEncoder': TransformerEncoder})
print(model.evaluate(int_test_ds))

Epoch 1/20
625/625 [==============================] - ETA: 0s - loss: 0.5042 - binary_accuracy: 0.7598INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 282s 451ms/step - loss: 0.5042 - binary_accuracy: 0.7598 - val_loss: 0.3555 - val_binary_accuracy: 0.8488
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.3425 - binary_accuracy: 0.8500INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 281s 450ms/step - loss: 0.3425 - binary_accuracy: 0.8500 - val_loss: 0.3120 - val_binary_accuracy: 0.8686
Epoch 3/20
625/625 [==============================] - 278s 444ms/step - loss: 0.3038 - binary_accuracy: 0.8723 - val_loss: 0.3136 - val_binary_accuracy: 0.8706
Epoch 4/20
625/625 [==============================] - ETA: 0s - loss: 0.2738 - binary_accuracy: 0.8867INFO:tensorflow:Assets written to: transformer_encoder/assets


INFO:tensorflow:Assets written to: transformer_encoder/assets


625/625 [==============================] - 279s 447ms/step - loss: 0.2738 - binary_accuracy: 0.8867 - val_loss: 0.2985 - val_binary_accuracy: 0.8726
Epoch 5/20
625/625 [==============================] - 281s 450ms/step - loss: 0.2447 - binary_accuracy: 0.8996 - val_loss: 0.2993 - val_binary_accuracy: 0.8806
Epoch 6/20
625/625 [==============================] - 278s 445ms/step - loss: 0.2116 - binary_accuracy: 0.9173 - val_loss: 0.3035 - val_binary_accuracy: 0.8800
Epoch 7/20
625/625 [==============================] - 282s 451ms/step - loss: 0.1843 - binary_accuracy: 0.9273 - val_loss: 0.2991 - val_binary_accuracy: 0.8860
Epoch 8/20
625/625 [==============================] - 284s 455ms/step - loss: 0.1519 - binary_accuracy: 0.9413 - val_loss: 0.3146 - val_binary_accuracy: 0.8852
Epoch 9/20
625/625 [==============================] - 280s 448ms/step - loss: 0.1231 - binary_accuracy: 0.9557 - val_loss: 0.3491 - val_binary_accuracy: 0.8778
Epoch 10/20
625/625 [==============================

In [35]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, seq_len, in_dim, out_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=in_dim, output_dim=out_dim)
        self.position_embeddings = layers.Embedding(input_dim=seq_len, output_dim=out_dim)
        self.seq_len = seq_len
        self.in_dim = in_dim
        self.out_dim = out_dim
    
    def call(self, inputs):
        positions = tf.range(start=0, limit=inputs.shape[-1])
        embeded_tokens = self.token_embeddings(inputs)
        embeded_positions = self.position_embeddings(positions)
        return embeded_tokens + embeded_positions
    
    def compute_mask(self, inputs, mark=None):
        return tf.math.not_equal(inputs, 0)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            'seq_len': self.seq_len,
            'in_dim': self.in_dim,
            'out_dim': self.out_dim
        })
        return config

In [37]:
vocab_size = 20000
seq_len = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = layers.Input(shape=(None,), dtype=tf.int64)
x = PositionalEmbedding(seq_len=seq_len, in_dim=vocab_size, out_dim=embed_dim)(inputs)
x = TransformerEncoder(embed_dim=embed_dim, dense_dim=dense_dim, num_heads=num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation=activations.sigmoid)(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer=optimizers.RMSprop(), loss=losses.BinaryCrossentropy(), metrics=[metrics.BinaryAccuracy()])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_1 (Po  (None, None, 256)         5273600   
 sitionalEmbedding)                                              
                                                                 
 transformer_encoder_5 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_5 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                           

In [38]:
callback_list = [
    callbacks.ModelCheckpoint('full_transformer_encoder', save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callback_list)

model = keras.models.load_model('full_transformer_encoder', custom_objects={
    'TransformerEncoder': TransformerEncoder,
    'PositionalEmbedding': PositionalEmbedding
})
print(model.evaluate(int_test_ds))

Epoch 1/20
625/625 [==============================] - ETA: 0s - loss: 0.5334 - binary_accuracy: 0.7369INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 296s 472ms/step - loss: 0.5334 - binary_accuracy: 0.7369 - val_loss: 0.3647 - val_binary_accuracy: 0.8426
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.3032 - binary_accuracy: 0.8723INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 297s 475ms/step - loss: 0.3032 - binary_accuracy: 0.8723 - val_loss: 0.2871 - val_binary_accuracy: 0.8790
Epoch 3/20
625/625 [==============================] - ETA: 0s - loss: 0.2386 - binary_accuracy: 0.9051INFO:tensorflow:Assets written to: full_transformer_encoder/assets


INFO:tensorflow:Assets written to: full_transformer_encoder/assets


625/625 [==============================] - 298s 476ms/step - loss: 0.2386 - binary_accuracy: 0.9051 - val_loss: 0.2830 - val_binary_accuracy: 0.8852
Epoch 4/20
625/625 [==============================] - 297s 475ms/step - loss: 0.1956 - binary_accuracy: 0.9247 - val_loss: 0.3101 - val_binary_accuracy: 0.8824
Epoch 5/20
625/625 [==============================] - 297s 476ms/step - loss: 0.1639 - binary_accuracy: 0.9360 - val_loss: 0.3955 - val_binary_accuracy: 0.8758
Epoch 6/20
625/625 [==============================] - 296s 474ms/step - loss: 0.1398 - binary_accuracy: 0.9470 - val_loss: 0.4381 - val_binary_accuracy: 0.8728
Epoch 7/20
625/625 [==============================] - 293s 469ms/step - loss: 0.1146 - binary_accuracy: 0.9564 - val_loss: 0.4788 - val_binary_accuracy: 0.8814
Epoch 8/20
625/625 [==============================] - 295s 471ms/step - loss: 0.0920 - binary_accuracy: 0.9659 - val_loss: 0.6312 - val_binary_accuracy: 0.8832
Epoch 9/20
625/625 [==============================]

In [40]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config
    

vocab_size = 20000
sequence_length = 600
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder_0",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=20, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder_0",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding})
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 positional_embedding_2 (Po  (None, None, 256)         5273600   
 sitionalEmbedding)                                              
                                                                 
 transformer_encoder_6 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_6 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                           

INFO:tensorflow:Assets written to: full_transformer_encoder_0/assets


625/625 [==============================] - 295s 471ms/step - loss: 0.5371 - accuracy: 0.7372 - val_loss: 0.3388 - val_accuracy: 0.8484
Epoch 2/20
625/625 [==============================] - ETA: 0s - loss: 0.3040 - accuracy: 0.8719INFO:tensorflow:Assets written to: full_transformer_encoder_0/assets


INFO:tensorflow:Assets written to: full_transformer_encoder_0/assets


625/625 [==============================] - 296s 474ms/step - loss: 0.3040 - accuracy: 0.8719 - val_loss: 0.2961 - val_accuracy: 0.8832
Epoch 3/20
625/625 [==============================] - 297s 475ms/step - loss: 0.2415 - accuracy: 0.9021 - val_loss: 0.4661 - val_accuracy: 0.8274
Epoch 4/20
625/625 [==============================] - 294s 471ms/step - loss: 0.1990 - accuracy: 0.9205 - val_loss: 0.3126 - val_accuracy: 0.8828
Epoch 5/20
625/625 [==============================] - ETA: 0s - loss: 0.1662 - accuracy: 0.9365INFO:tensorflow:Assets written to: full_transformer_encoder_0/assets


INFO:tensorflow:Assets written to: full_transformer_encoder_0/assets


625/625 [==============================] - 295s 473ms/step - loss: 0.1662 - accuracy: 0.9365 - val_loss: 0.2945 - val_accuracy: 0.8896
Epoch 6/20
625/625 [==============================] - 295s 473ms/step - loss: 0.1397 - accuracy: 0.9474 - val_loss: 0.3706 - val_accuracy: 0.8842
Epoch 7/20
625/625 [==============================] - 294s 470ms/step - loss: 0.1192 - accuracy: 0.9571 - val_loss: 0.4080 - val_accuracy: 0.8886
Epoch 8/20
625/625 [==============================] - 295s 471ms/step - loss: 0.0988 - accuracy: 0.9658 - val_loss: 0.5944 - val_accuracy: 0.8634
Epoch 9/20
625/625 [==============================] - 296s 473ms/step - loss: 0.0767 - accuracy: 0.9725 - val_loss: 1.5398 - val_accuracy: 0.8120
Epoch 10/20
625/625 [==============================] - 294s 471ms/step - loss: 0.0626 - accuracy: 0.9789 - val_loss: 0.5613 - val_accuracy: 0.8664
Epoch 11/20
625/625 [==============================] - 295s 472ms/step - loss: 0.0461 - accuracy: 0.9848 - val_loss: 0.9259 - val_accu